In [ ]:
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch

from pathlib import Path
from fastai.callbacks import *
from fastai.vision import *

In [ ]:
# 指定数据集路径
path = Path('/kaggle/input/mri_dataset/MRI_Dataset/')
fnames = get_image_files(path / 'image')
lbl_names = get_image_files(path / 'label')

In [ ]:
# 输出当前图像的路径
img_f = fnames[1020]
print(img_f)

# 显示当前图像
img = open_image(img_f)
img.show(figsize=(5, 5))

In [ ]:
# 获取当前图像的label的路径
get_y_fn = lambda x : path / 'label' / f'{x.stem}_mask{x.suffix}'
print(get_y_fn(img_f))

In [ ]:
# 显示当前图像的label
mask = open_mask(get_y_fn(img_f), div=True)
mask.show(figsize=(5, 5), alpha=1)

In [ ]:
# 图像及其label的读取，预处理（归一化）
src_size = np.array(mask.shape[1:])
size = src_size # 图像尺寸 (256, 256)
bs = 32     # batch size

class MySegmentationLabelList(SegmentationLabelList):
    def open(self, fn):
        return open_mask(fn, div=True)

class MySegmentationItemList(SegmentationItemList):
    _label_cls, _square_show_res = MySegmentationLabelList, False

data = (MySegmentationItemList
        .from_folder(path / 'image')
        .split_by_rand_pct(0.2)
        .label_from_func(get_y_fn, classes=['0', '1'])
        .transform(get_transforms(), tfm_y=True, size=size)
        .databunch(bs=bs, path='/kaggle')
        .normalize(imagenet_stats))

In [ ]:
# 显示部分数据
data.show_batch(3, figsize=(8, 8), alpha=0.5)

In [ ]:
# 定义模型为以resnet34为前端的unet模型
learn = unet_learner(data, models.resnet34, self_attention=True, metrics=dice, wd=1e-2).to_fp16()

In [ ]:
# 寻找最佳学习率
learn.lr_find(num_it=200)
learn.recorder.plot(suggestion=True)

In [ ]:
# 训练模型
learn.fit_one_cycle(1, max_lr=slice(6.92e-04, 1e-03), pct_start=0.7,
                    callbacks=[SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='model')])

In [ ]:
# 绘制loss曲线
learn.recorder.plot_losses()

In [ ]:
# 绘制指标曲线（dice系数）
learn.recorder.plot_metrics()

In [ ]:
# 绘制学习率曲线
learn.recorder.plot_lr()

In [ ]:
# 加载训练好的最优模型，并解冻所有层。之后将对模型进行微调
learn.load('model')
learn = learn.to_fp16()
learn.unfreeze()

In [ ]:
# 寻找最佳学习率
learn.lr_find(start_lr=1e-9, end_lr=1e-4, num_it=200)
learn.recorder.plot(suggestion=True)

In [ ]:
# 使用小学习率进行模型微调
learn.fit_one_cycle(5, max_lr=slice(6.68e-06, 8e-09), pct_start=0.8,
                    callbacks=[SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='model')])

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.recorder.plot_metrics()

In [ ]:
learn.recorder.plot_lr()

In [ ]:
# 加载微调后的最佳模型，并展示预测结果
learn.load('model')
learn = learn.to_fp16()
learn.show_results(alpha=0.6)